<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/ESTCScrape_chromedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#source: chatGPT and stack overflow

In [2]:
!pip install selenium --quiet
!pip install pandas --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [3]:
pip install --upgrade selenium chromedriver-autoinstaller --quiet

In [6]:
###### ATTTEMPT 1 (FAIL) (year search included in the URL)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Set up Selenium with Chrome WebDriver
# #used StackOverflow to help adjust https://stackoverflow.com/questions/76461596/unable-to-use-selenium-webdriver-getting-two-exceptions
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
# got the items info from the HTML code on inspect element
def scrape_record(record_url):
    driver.get(record_url)
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
        )
        alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
        author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
        city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
        country_code = driver.find_element(By.XPATH, '//td[text()="Country Code"]/following-sibling::td').text
        estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
        format = driver.find_element(By.XPATH, '//td[text()="Format"]/following-sibling::td').text
        imprint_original = driver.find_element(By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td').text
        language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
        pagination = driver.find_element(By.XPATH, '//td[text()="Pagination"]/following-sibling::td').text
        title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
        year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

        return {
            'Alt Title': alt_title,
            'Author': author,
            'City': city,
            'Country Code': country_code,
            'ESTC No': estc_no,
            'Format': format,
            'Imprint Original': imprint_original,
            'Language Code': language_code,
            'Pagination': pagination,
            'Title': title,
            'Year': year
        }
    except NoSuchElementException as e:
        print(f"Element not found in record {record_url}: {e}")
        return None
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None

# Function to scrape records for a given year
def scrape_year(year):
    records = []
    search_url = f"{base_url}search?q=&year={year}&year_end={year}"
    print(f"Navigating to search URL: {search_url}")
    driver.get(search_url)

    while True:
        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
            )
            record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")
            record_urls = [element.get_attribute('href') for element in record_elements]

            print(f"Found {len(record_urls)} records on the page.")

            for record_url in record_urls:
                print(f"Scraping record: {record_url}")
                record_data = scrape_record(record_url)
                if record_data:
                    records.append(record_data)

            # Check if there is a next page
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
                if 'disabled' in next_button.get_attribute('class'):
                    print("Next button is disabled. No more pages.")
                    break
                else:
                    print("Clicking next button to go to the next page.")
                    next_button.click()
                    time.sleep(10)
            except NoSuchElementException:
                print("Next button not found. Assuming no more pages.")
                break
        except TimeoutException:
            print("Timed out waiting for records to load. Moving to the next page.")
            break

    return records

### TEST 1473-1475 #####
# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records1473_147.csv', index=False)
df.to_excel('ESTC_records1473_147.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/search?q=&year=1473&year_end=1473
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Navigating to search URL: https://estc.printprobability.org/search?q=&year=1474&year_end=1474
Timed out waiting for records to load. Moving to the next page.


In [7]:
###### ATTTEMPT 2 (FAIL) (trying to extract some records from the view record buttom - its not finding the records)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Set up Selenium with Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_element):
    """
    Scrape data from a single record element.
    """
    try:
        # Extract data from the record element
        author = record_element.find_element(By.CSS_SELECTOR, 'div.authors').text.strip()
        estc_no = record_element.find_element(By.CSS_SELECTOR, 'div.estcNo').text.strip()
        year = record_element.find_element(By.CSS_SELECTOR, 'div.year').text.strip()

        return {
            'Author': author,
            'ESTC No': estc_no,
            'Year': year
        }
    except Exception as e:
        print(f"Error scraping record: {e}")
        return None

# Function to scrape records from the current page
def scrape_current_page():
    """
    Scrape records from the current page.
    """
    records = []
    try:
        record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record")
        print(f"Found {len(record_elements)} records on the page.")

        for record_element in record_elements:
            record_data = scrape_record(record_element)
            if record_data:
                records.append(record_data)

        return records
    except Exception as e:
        print(f"Error scraping current page: {e}")
        return None

# Function to check if there's a next page and navigate to it
def go_to_next_page():
    """
    Check if there's a next page and navigate to it.
    """
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
        if 'disabled' not in next_button.get_attribute('class'):
            print("Clicking next button to go to the next page.")
            next_button.click()
            time.sleep(2)
            return True
        else:
            print("Next button is disabled. No more pages.")
            return False
    except NoSuchElementException:
        print("Next button not found. Assuming no more pages.")
        return False

### MAIN SCRIPT ###

# Navigate to the base URL
driver.get(base_url)

# Scrape records from the main page and navigate to the next page if available
all_records = []
while True:
    print("Scraping records from the current page.")
    records = scrape_current_page()
    if records:
        all_records.extend(records)

    if not go_to_next_page():
        break

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records_main_page.csv', index=False)
df.to_excel('ESTC_records_main_page.xlsx', index=False)

# Close the driver
driver.quit()


Scraping records from the current page.
Found 0 records on the page.
Next button not found. Assuming no more pages.


In [8]:
###### ATTTEMPT 3 (FAIL) (trying to identify where time out occurs)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Set up Selenium with Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_url):
    """
    Scrape data from a single record URL.
    """
    driver.get(record_url)
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
        )
        # Extract data from the record page
        alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
        author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
        city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
        country_code = driver.find_element(By.XPATH, '//td[text()="Country Code"]/following-sibling::td').text
        estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
        format = driver.find_element(By.XPATH, '//td[text()="Format"]/following-sibling::td').text
        imprint_original = driver.find_element(By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td').text
        language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
        pagination = driver.find_element(By.XPATH, '//td[text()="Pagination"]/following-sibling::td').text
        title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
        year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

        return {
            'Alt Title': alt_title,
            'Author': author,
            'City': city,
            'Country Code': country_code,
            'ESTC No': estc_no,
            'Format': format,
            'Imprint Original': imprint_original,
            'Language Code': language_code,
            'Pagination': pagination,
            'Title': title,
            'Year': year
        }
    except NoSuchElementException as e:
        print(f"Element not found in record {record_url}: {e}")
        return None
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None

# Function to scrape records for a given year
def scrape_year(year):
    """
    Scrape records for a given year.
    """
    records = []
    driver.get(base_url)

    print(f"Navigating to search URL: {base_url} Year search: {year}")

    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ais-Dropdown-button"))
        )

        # Click the "Year" button to open the dropdown
        print("Clicking the 'Year' button")
        year_button = driver.find_element(By.CLASS_NAME, "ais-Dropdown-button")
        year_button.click()

        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ais-RangeInput-input--min"))
        )

        # Input the start year
        print(f"Inputting the start year: {year}")
        start_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--min")
        start_year_input.clear()
        start_year_input.send_keys(str(year))

        # Input the end year
        print(f"Inputting the end year: {year}")
        end_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--max")
        end_year_input.clear()
        end_year_input.send_keys(str(year))

        # Click the "Go" button
        print("Clicking the 'Go' button")
        go_button = driver.find_element(By.CLASS_NAME, "ais-RangeInput-submit")
        go_button.click()

        while True:
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
            )
            record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")

            if not record_elements:
                print(f"No record elements found on the page for year {year}.")
                break

            record_urls = [element.get_attribute('href') for element in record_elements]
            print(f"Found {len(record_urls)} records on the page.")

            for record_url in record_urls:
                print(f"Scraping record: {record_url}")
                record_data = scrape_record(record_url)
                if record_data:
                    records.append(record_data)

            # Check if there is a next page
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
                if 'disabled' in next_button.get_attribute('class'):
                    print("Next button is disabled. No more pages.")
                    break
                else:
                    print("Clicking next button to go to the next page.")
                    next_button.click()
                    time.sleep(2)
            except NoSuchElementException:
                print("Next button not found. Assuming no more pages.")
                break
    except TimeoutException:
        print("Timed out waiting for records to load. Moving to the next page.")

    return records

### TEST 1473-1475 #####
# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records1473_1474newfunctionscrape.csv', index=False)
df.to_excel('ESTC_records1473_1474newfunctionscrape.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/ Year search: 1473
Clicking the 'Year' button
Inputting the start year: 1473
Inputting the end year: 1473
Clicking the 'Go' button
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Navigating to search URL: https://estc.printprobability.org/ Year search: 1474
Clicking the 'Year' button
Inputting the start year: 1474
Inputting the end year: 1474
Clicking the 'Go' button
Timed out waiting for records to load. Moving to the next page.
